# Installing necessary libraries

In [1]:
!pip install pyathena

In [2]:
!pip install awswrangler

In [3]:
!pip install seaborn

In [4]:
!pip3 install -U sagemaker

## Importing necessary libraries

In [5]:
from pyathena import connect

In [6]:
import awswrangler as wr
import pandas as pd

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

## Setup Sagemaker Instance

In [8]:
import os
import boto3
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

bucket = sess.default_bucket()
prefix = "instacart-xgboost"

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [9]:
import pandas as pd
import numpy as np

s3 = boto3.client("s3")

# Converting InstantCart CSV dataset into Parquet

In [10]:
filename = "filtered_frequent_buyers_v1.csv"
csv_path = "filtered_frequent_buyers_v1.csv"
df = pd.read_csv(csv_path)

# Loading CSV from pandas and split 40% for training data
train_df, remaining_df = train_test_split(df, train_size=0.4, random_state=42)

# Split the remaining data into production and temp datasets (66% production, 33% temp)
production_df, temp_df = train_test_split(remaining_df, train_size=0.666666, random_state=42)

# Split the temp data into test and validation datasets (50% test, 50% validation)
test_df, validation_df = train_test_split(temp_df, train_size=0.5, random_state=42)

In [ ]:
# Defining S3 paths
parquet_output_path = "s3://{}/data/parquet".format(bucket)

# Save each split dataset to Parquet with partitioning by 'department'
wr.s3.to_parquet(
    df=train_df,
    path=parquet_output_path + "train/",
    dataset=True,
    mode="overwrite",
    partition_cols=["department"],
    compression="snappy"
)

wr.s3.to_parquet(
    df=production_df,
    path=parquet_output_path + "production/",
    dataset=True,
    mode="overwrite",
    partition_cols=["department"],
    compression="snappy"
)

wr.s3.to_parquet(
    df=test_df,
    path=parquet_output_path + "test/",
    dataset=True,
    mode="overwrite",
    partition_cols=["department"],
    compression="snappy"
)

wr.s3.to_parquet(
    df=validation_df,
    path=parquet_output_path + "validation/",
    dataset=True,
    mode="overwrite",
    partition_cols=["department"],
    compression="snappy"
)

[02/05/25 01:13:06] INFO     Initializing a Ray instance                                               ]8;id=730438;file:///opt/conda/lib/python3.11/site-packages/awswrangler/distributed/ray/_core.py\_core.py]8;;\:]8;id=960625;file:///opt/conda/lib/python3.11/site-packages/awswrangler/distributed/ray/_core.py#202\202]8;;\

2025-02-05 01:13:10,476	INFO worker.py:1786 -- Started a local Ray instance.


In [ ]:
# Create the database if it doesn't exist
database_name = "instacart_db"
try:
    # Create the database in AWS Glue
    wr.catalog.create_database(name=database_name)
    print(f"Database '{database_name}' created successfully!")
except Exception as e:
    print(f"Error creating database: {e}")

In [ ]:
# Register the Parquet tables in AWS Glue
table_name = "instacart_orders"

wr.catalog.create_parquet_table(
    database=database_name,
    table=table_name,
    path=parquet_output_path + "train/",
    columns_types={
        "order_id": "bigint",
        "product_id": "bigint",
        "add_to_cart_order": "int",
        "reordered": "int",
        "user_id": "bigint",
        "product_name": "string",
        "aisle_id": "int",
        "department_id": "int",
        "aisle": "string",
        "department": "string"
    },
    partitions_types={"department": "string"},
    description="Partitioned Instacart orders dataset for optimized Athena queries."
)

print("Partitioned Parquet table registered in AWS Glue successfully.")


# Setting up Database for InstantCart

In [ ]:
from pyathena import connect

# Defineing AWS Resources
bucket_name = bucket
region = "us-east-1"
database_name = "instacart_db"
table_name = "instacart_orders"
s3_data_location = f"s3://{bucket_name}/data-lake/project/partitioned/train/"  # Using partitioned dataset

# Defining Athena Staging Directory
s3_staging_dir = f"s3://{bucket_name}/athena/instacart_staging/"

# Creating Athena Connection
try:
    conn = connect(s3_staging_dir=s3_staging_dir, region_name=region)
    cursor = conn.cursor()
    print("Connected to Athena successfully.")
except Exception as e:
    print("Error connecting to Athena:", e)

# Creating Database
create_db_query = f"CREATE DATABASE IF NOT EXISTS {database_name}"
cursor.execute(create_db_query)
print(f"Database '{database_name}' created successfully!")

# Verifying Database Creation
cursor.execute("SHOW DATABASES")
databases = [row[0] for row in cursor.fetchall()]
if database_name in databases:
    print(f"Database '{database_name}' exists!")


# Creating Athena database

In [ ]:
create_table_query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name} (
    order_id BIGINT,
    product_id BIGINT,
    add_to_cart_order INT,
    reordered INT,
    user_id BIGINT,
    product_name STRING,
    aisle_id INT,
    department_id INT,
    aisle STRING
)
PARTITIONED BY (department STRING)  -- Partitioned by 'department'
STORED AS PARQUET
LOCATION 's3://{bucket_name}/data-lake/project/partitioned/train/'  
TBLPROPERTIES ('parquet.compression'='SNAPPY');
"""

# Execute Table Creation Query
cursor.execute(create_table_query)
print(f"Table '{table_name}' created successfully in database '{database_name}'.")

In [ ]:
# Running MSCK REPAIR to Load Partitions
cursor.execute(f"MSCK REPAIR TABLE {database_name}.{table_name}")
print("Partitions updated successfully.")

In [ ]:
cursor.execute("SHOW DATABASES")
databases = [row[0] for row in cursor.fetchall()]
if database_name in databases:
    print(f"Database '{database_name}' exists in Athena!")
else:
    print(f"Database '{database_name}' does not exist.")

In [ ]:
import boto3

s3 = boto3.client('s3')
response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
for obj in response.get('Contents', []):
    print(obj['Key'])

# Test

In [ ]:
from pyathena import connect
import pandas as pd

In [ ]:
statement = "SELECT * FROM instacart_orders;"

In [ ]:
df = pd.read_sql(statement, conn)

# Not Working

In [ ]:
# Running a Sample Query to Verify Data
test_query = f"SELECT count(*) FROM {database_name}.{table_name} ;"
cursor.execute(test_query)
rows = cursor.fetchall()

print("Sample Query Results:")
for row in rows:
    print(row)

In [ ]:
cursor.execute(f"SHOW PARTITIONS {database_name}.{table_name}")
partitions = cursor.fetchall()
if partitions:
    print(f"Partitions found in table '{table_name}': {partitions}")
else:
    print(f"No partitions found in table '{table_name}'.")


# Exploratory Data Analysis

## Checking the Total Orders and Unique Users

In [ ]:
# First Query: Total Orders & Unique Users
query = f"""
SELECT
    COUNT(DISTINCT order_id) AS total_orders,
    COUNT(DISTINCT user_id) AS unique_users
FROM {database_name}.{table_name};
"""

# Executing query
cursor.execute(query)
rows = cursor.fetchall()

# Printing results
print("Total Orders & Unique Users:")
for row in rows:
    print(row)


## Top 10 Most Ordered Products


In [ ]:
query = f"""
SELECT product_name, COUNT(*) AS total_orders
FROM {database_name}.{table_name}
GROUP BY product_name
ORDER BY total_orders DESC
LIMIT 10;
"""
cursor.execute(query)
rows = cursor.fetchall()
print("Top 10 Most Ordered Products:")
for row in rows:
    print(row)


In [ ]:
# ===========================
#  Bar Chart: Top 10 Most Ordered Products
# ===========================

products = ["Banana", "Bag of Organic Bananas", "Organic Strawberries", "Organic Hass Avocado",
            "Organic Baby Spinach", "Organic Raspberries", "Organic Avocado", "Organic Whole Milk",
            "Limes", "Large Lemon"]
total_orders = [52073, 46964, 39311, 31555, 29459, 21479, 19582, 19211, 17136, 16719]

# Creating DataFrame
df_products = pd.DataFrame({"Product Name": products, "Total Orders": total_orders})

# Plotting Bar Chart
plt.figure(figsize=(12, 6))
sns.barplot(x="Total Orders", y="Product Name", data=df_products, palette="viridis")
plt.xlabel("Total Orders")
plt.ylabel("Product Name")
plt.title("Top 10 Most Ordered Products")
plt.show()


### Insights

- Bananas are the most ordered item with 52073 orders.
- Organic produce is extremely popular. Where 7 out of 10 items are organic.
- Dairy products like Whole Milk rank in the Top 10.
- Cirtrus fruits like Limes and Lemons are in high demand.

## Reorder Rate per Product

In [ ]:
query = f"""
SELECT
    product_name,
    COUNT(*) AS total_orders,
    SUM(reordered) AS total_reorders,
    ROUND(100.0 * SUM(reordered) / COUNT(*), 2) AS reorder_rate
FROM {database_name}.{table_name}
GROUP BY product_name
ORDER BY reorder_rate DESC
LIMIT 10;
"""
cursor.execute(query)
rows = cursor.fetchall()
print("Top 10 Products with Highest Reorder Rate:")
for row in rows:
    print(row)


### Insights for Top 10 Products with Highest Reorder Rate:

- **100% Lactose-Free Milk** has a **100% reorder rate**, indicating it is a highly demanded product among lactose-intolerant customers who consistently reorder it.
- **Salted Sweet Cream Butter Quarters**, **Sparkling Water Bottles**, and **Premium Lots of Pulp Orange Juice** also show a **100% reorder rate**, signifying that these are popular products with strong customer loyalty and recurring demand.
- **Classic Baby Creamers Potatoes** and **Peru Sweet Onions** are products with **100% reorder rate**, pointing to their continued preference and high re-purchase frequency among customers.
- **Thirst Quencher Caffeine-Free Naturally Flavored Citrus Soda** and **Smoked Whitefish Salad** reflect **100% reorder rates**, highlighting that these items are consistently chosen by customers who keep coming back for more.
- **Seltzer Water** and **Green Bananas** demonstrate high reorder demand, each maintaining **100% reorder rate**, suggesting these are staple items that customers rely on regularly.

## Orders by Department

In [ ]:
query = f"""
SELECT department, COUNT(*) AS total_orders
FROM {database_name}.{table_name}
GROUP BY department
ORDER BY total_orders DESC;
"""
cursor.execute(query)
rows = cursor.fetchall()
print("Orders by Department:")
for row in rows:
    print(row)


In [ ]:
# ===========================
# Bar Chart: Orders by Department
# ===========================

# Data from previous query (Department, Total Orders)
departments = ["produce", "dairy eggs", "snacks", "beverages", "frozen",
               "pantry", "bakery", "deli", "canned goods", "dry goods pasta"]
total_orders = [1116850, 619999, 294213, 234613, 201359, 138924, 127884, 114349, 102416, 84651]

# Creating DataFrame
df_orders_by_department = pd.DataFrame({"Department": departments, "Total Orders": total_orders})

# Plotting Bar Chart
plt.figure(figsize=(12, 6))
sns.barplot(x="Total Orders", y="Department", data=df_orders_by_department, palette="magma")
plt.xlabel("Total Orders")
plt.ylabel("Department")
plt.title("Total Orders by Department")
plt.show()


### Insights

- Produce is the most ordered department with ~1.11 million orders. This aligns with the earlier Top Ordered Products (bananas, avocados, berries). On another hand, Fresh produce is frequently bought and likely reordered often.
- Dairy & Eggs ranks second with 619K orders.
- Snacks are the third most popular category (294K orders).
    - Expect chips, granola bars, and nuts to dominate.
- Beverages & Frozen Foods also have strong demand.
    - Beverages (~234K orders) likely include popular items like bottled water, juices, and coffee.
    - Frozen Foods (~201K orders) suggest customers stock up on frozen essentials.
- Pantry Staples, Bakery, and Deli also contribute significantly. It could likely contain bread, canned goods, and dry pasta.

## Most Popular Aisles

In [ ]:
query = f"""
SELECT aisle, COUNT(*) AS total_orders
FROM {database_name}.{table_name}
GROUP BY aisle
ORDER BY total_orders DESC
LIMIT 10;
"""
cursor.execute(query)
rows = cursor.fetchall()
print("Top 10 Aisles:")
for row in rows:
    print(row)


In [ ]:
# ===========================
# Bar Chart: Most Popular Aisles
# ===========================

# Data from previous query (Aisle, Total Orders)
aisles = ["fresh fruits", "fresh vegetables", "packaged vegetables fruits", "yogurt",
          "packaged cheese", "milk", "water seltzer sparkling water", "chips pretzels",
          "soy lactosefree", "bread"]
aisle_orders = [450026, 404252, 205492, 184903, 112690, 103953, 78019, 74536, 68786, 65469]

# Creating DataFrame
df_aisles = pd.DataFrame({"Aisle": aisles, "Total Orders": aisle_orders})

# Plotting Bar Chart
plt.figure(figsize=(12, 6))
sns.barplot(x="Total Orders", y="Aisle", data=df_aisles, palette="coolwarm")
plt.xlabel("Total Orders")
plt.ylabel("Aisle")
plt.title("Top 10 Most Popular Aisles")
plt.show()

### Insights
- Fresh Produce Dominance:
    - Fresh Fruits (450K orders) and Fresh Vegetables (404K orders) are the top two aisles.
    - Combined, these two alone account for over 950K orders, which reinforces why Produce is the top department.

- Dairy is Highly Popular:
    - Yogurt (184K orders) and Packaged Cheese (112K orders) show strong demand.
    - Milk (103K orders) further confirms that dairy products are household essentials.

- Beverages are a Major Category:
    - Water, Seltzer, and Sparkling Water (78K orders) ranks #7, showing strong demand for bottled drinks.

- Snacks & Bread are Key Pantry Items
    - Chips & Pretzels (74K orders) are among the most frequently purchased snacks.
    - Bread (65K orders) confirms why Bakery is among the top departments.

- Plant-Based Alternatives are Growing
    - Soy & Lactose-Free Products (68K orders) indicate increased demand for dairy-free alternatives.

##  Reorder Ratio by Department

In [ ]:
query = f"""
SELECT
    department,
    COUNT(*) AS total_orders,
    SUM(reordered) AS total_reorders,
    ROUND(100.0 * SUM(reordered) / COUNT(*), 2) AS reorder_ratio
FROM {database_name}.{table_name}
GROUP BY department
ORDER BY reorder_ratio DESC;
"""
cursor.execute(query)
rows = cursor.fetchall()
print("Reorder Ratio by Department:")
for row in rows:
    print(row)


### Insights

- Dairy & Eggs Have the Highest Reorder Rate (82.73%)
    - Most frequently reordered category.
    - Milk, Yogurt, and Cheese are household staples → high repurchase behavior.

- Beverages Rank #2 in Reorders (80.85%)
    - Bottled Water, Sparkling Water, and Coffee/Tea are commonly repurchased.
    - These items are frequently consumed & replaced regularly.

- Produce Has a High Reorder Rate (80.71%)
    - Fruits and vegetables have a high purchase frequency.
    - Bananas, Avocados, and Berries from previous queries reinforce this trend.

- Bakery (80.43%) & Deli (78.53%) Show Strong Reorder Loyalty
    - Bread, Bagels, and Pre-packaged Deli Items are regularly bought items.
    - Customers often stick to the same brands.

- Snacks & Frozen Foods Have Moderate Reorder Rates (~70%)
    - Chips, Pretzels, and Frozen Meals are repurchased but less frequently than fresh foods.

- Canned Goods & Dry Goods Have Lower Reorder Rates (~65%)
    - Longer shelf life → not purchased as frequently.

- Pasta, sauces, and canned vegetables last longer → lower immediate repurchase need.

- Pantry Has the Lowest Reorder Rate (57.26%)
    - Less frequent purchases of pantry staples like flour, condiments, and spices.

In [ ]:
# ===========================
# Pie Chart: Reorder Ratio by Department
# ===========================

# Data from previous query (Department, Reorder Ratio)
departments = ["dairy eggs", "beverages", "produce", "bakery", "deli",
               "snacks", "frozen", "canned goods", "dry goods pasta", "pantry"]
reorder_ratio = [82.73, 80.85, 80.71, 80.43, 78.53, 74.01, 71.87, 65.51, 65.33, 57.26]

# Create DataFrame
df_departments = pd.DataFrame({"Department": departments, "Reorder Ratio": reorder_ratio})

# Plot Pie Chart
plt.figure(figsize=(10, 6))
plt.pie(df_departments["Reorder Ratio"], labels=df_departments["Department"],
        autopct="%1.1f%%", colors=sns.color_palette("viridis", len(departments)), startangle=140)
plt.title("Reorder Ratio by Department")
plt.axis("equal")  # Equal aspect ratio ensures the pie is drawn as a circle
plt.show()

In [ ]:
# ===========================
# Stacked Bar Chart: Reordered vs Non-Reordered Orders by Department
# ===========================

# Data from previous query (Department, Reordered Orders)
departments = ["dairy eggs", "beverages", "produce", "bakery", "deli",
               "snacks", "frozen", "canned goods", "dry goods pasta", "pantry"]
total_orders = [619999, 234613, 1116850, 127884, 114349, 294213, 201359, 102416, 84651, 138924]
reordered = [512956, 189692, 901358, 102853, 89802, 217758, 144719, 67094, 55299, 79541]
non_reordered = [total - reorder for total, reorder in zip(total_orders, reordered)]

# Create DataFrame
df_reorders = pd.DataFrame({"Department": departments, "Reordered": reordered, "Non-Reordered": non_reordered})

# Plot Stacked Bar Chart (Reordered vs Non-Reordered)
df_reorders.set_index("Department")[["Reordered", "Non-Reordered"]].plot(kind="bar", stacked=True, figsize=(12, 6), colormap="viridis")
plt.xlabel("Department")
plt.ylabel("Number of Orders")
plt.title("Reordered vs Non-Reordered Orders by Department")
plt.legend(["Reordered", "Non-Reordered"])
plt.xticks(rotation=45)
plt.show()

**Analysis for reordered vs non-reordered orders by department**:
    
1. Produce has the highest number of orders overall

    - Most of these orders are reorders, confirming that fruits and vegetables are frequently repurchased.

2. Dairy & Eggs have the highest reorder percentage
    - Consistently repurchased products like milk, cheese, and yogurt drive these numbers.

3. Snacks and Frozen Foods have moderate reorder levels
    - Customers repurchase snacks and frozen goods but at a slightly lower frequency than fresh items.

4. Pantry and Canned Goods have the lowest reorder rates
    - These products have a longer shelf life, reducing the need for frequent repurchasing.

## Correlation Analysis

In [ ]:
# Data: Numerical Features for Correlation Analysis
departments = ["dairy eggs", "beverages", "produce", "bakery", "deli",
               "snacks", "frozen", "canned goods", "dry goods pasta", "pantry"]
total_orders = [619999, 234613, 1116850, 127884, 114349, 294213, 201359, 102416, 84651, 138924]
reordered = [512956, 189692, 901358, 102853, 89802, 217758, 144719, 67094, 55299, 79541]
reorder_ratio = [82.73, 80.85, 80.71, 80.43, 78.53, 74.01, 71.87, 65.51, 65.33, 57.26]

# Creating DataFrame
df_correlation = pd.DataFrame({
    "Total Orders": total_orders,
    "Reordered Orders": reordered,
    "Reorder Ratio (%)": reorder_ratio
})

# Computing Correlation Matrix
correlation_matrix = df_correlation.corr()

# Plotting Correlation Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix of Order Features")
plt.show()


- Total Orders & Reordered Orders (Correlation = 1.00)
    - This confirms that departments with high total orders also have high reorders.
    - Produce, Dairy, and Beverages are likely contributing to this trend.

- Total Orders & Reorder Ratio (Correlation = 0.49)
    - Moderate positive correlation → Higher orders somewhat influence the reorder rate, but not always.
    - Some departments may have high first-time purchases but lower reorder rates (e.g., snacks, pantry items).

- Reordered Orders & Reorder Ratio (Correlation = 0.52)
    - A moderate correlation suggests that higher reorder volumes influence reorder ratio but not perfectly.
    - Some products (like dairy & produce) are reordered very frequently, while others (like pantry goods) less frequently.


# Feature Engineering

For creating ML Feature features, I am going to use the following labels:

- User ID: 	This tracks individual purchase behavior.
- Product ID: Helps identify frequently reordered products.
- Department ID: Some departments have higher reorder rates.
- Aisle ID: Aisle-level trends impact reorder likelihood.
- Total Orders: Highly correlated with reorder behavior.
- Reorder Ratio: Strong predictor of repeat purchases.
- Total items in Orders: 	Determines if larger orders influence reorders.
- User Order Frequency: Identifies frequent vs. occasional buyers.
- User Reorder Percentage: Determines likelihood of repeat purchases.
- Product popularity: Captures demand for the product.
- Department reorder ratio: Some departments have stronger reorder trends.
- Aisle Reorder ratio: Aisle-specific reorder behavior.
- Product Reorder trend:  Helps detect seasonal or trending products.


Strongest predictions for EDA:

1. Reorder Ratio: strongly correlated with reorders.
2. Total Orders: high correlation with reorder likelihood.
3. User Reorder percerntage: Helps preduct if a user is likely to reorder.
4. Product Popularity: popular items have higher reorders.
5. Department and Aisle reorder ratios: certain categories drive higher reorders.
